In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, losses, Model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpi
import os

base_dir = '/home/gus/TensorFlow/catvsdog'

train_dir = os.path.join(base_dir,'train')
# validation_dir = os.path.join(base_dir, 'validation')

# train_cat_dir = os.path.join(train_dir, 'cat')
# train_dog_dir = os.path.join(train_dir, 'dog')


# validation_cats_dir = os.path.join(validation_dir, 'cats')
# validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# train_cat_fnames = os.listdir(train_cat_dir)
# train_dog_fnames = os.listdir(train_dog_dir)

# 3000개

# nrows, ncols = 4, 4
# pic_index = 0

# fig = plt.gcf()
# fig.set_size_inches(ncols*3, nrows*3)

# pic_index+=8

# next_cat_pix = [os.path.join(train_cat_dir, fname)
#                 for fname in train_cat_fnames[ pic_index-8:pic_index]]

# next_dog_pix = [os.path.join(train_dog_dir, fname)
#                 for fname in train_dog_fnames[ pic_index-8:pic_index]]

# for i, img_path in enumerate(next_cat_pix+next_dog_pix):
#   sp = plt.subplot(nrows, ncols, i + 1)
#   sp.axis('Off')

#   img = mpi.imread(img_path)
#   plt.imshow(img)

# plt.show()

#32 100 100 3 cat dog shuffled
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size = 32,
                                                   class_mode = 'binary',
                                                   target_size = (224,224))
# i = 0
# for X,Y in enumerate(train_generator):

# modeling
inp = layers.Input(shape=(224,224,3))
def pre_layer(x):
    x = layers.Conv2D(64,7,strides=2,padding='same',activation='relu')(x)
    x = layers.MaxPooling2D(3,strides=2)(x)
    x = layers.Conv2D(64,1,strides=1,padding='same',activation='relu')(x)
    x = layers.Conv2D(192,3,strides=1,padding='same',activation='relu')(x)
    x = layers.MaxPooling2D(3,strides=2)(x)
    return x
def inception(x,filters1,filters3_reduce,filters3,filters5_reduce,filters5,filterspool):
    path1 = layers.Conv2D(filters1,(1,1),padding='same',activation='relu')(x)
    path2 = layers.Conv2D(filters3_reduce,(1,1),padding='same',activation='relu')(x)
    path2 = layers.Conv2D(filters3,(1,1),padding='same',activation='relu')(path2)
    path3 = layers.Conv2D(filters5_reduce,(1,1),padding='same',activation='relu')(x)
    path3 = layers.Conv2D(filters5,(1,1), padding='same',activation='relu')(path3)
    path4 = layers.MaxPool2D((3,3),strides=(1,1),padding='same')(x)
    path4 = layers.Conv2D(filterspool,(1,1),padding='same',activation='relu')(path4)
    return tf.concat([path1, path2, path3, path4], axis=3)
def aux(x):
    aux = layers.AveragePooling2D((5,5),strides=3)(x)
    aux = layers.Conv2D(128,1,padding='same',activation='relu')(aux)
    aux = layers.Flatten()(aux)
    aux = layers.Dense(1024,activation='relu')(aux)
    aux = layers.Dropout(0.7)(aux)
    aux = layers.Dense(2,activation='softmax')(aux)
    return aux
def end_layer(x):
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(2, activation='softmax')(x)
    return x
x = pre_layer(inp)
x = inception(x,64,96,128,16,32,32)
x = inception(x,128,128,192,32,96,64)
x = layers.MaxPooling2D(3,strides=2)(x)
x = inception(x,192,96,208,16,48,64)
aux1 = aux(x)
x = inception(x,160,112,224,24,64,64)
x = inception(x,128,128,256,24,64,64)
aux2 = aux(x)
x = inception(x,256,160,320,32,128,128)
x = layers.MaxPooling2D(3,strides=2)(x)
x = inception(x,256,160,320,32,128,128)
x = inception(x,384,192,384,48,128,128)
out = end_layer(x)
model = Model(inputs=inp,outputs=[out,aux1,aux2])
def sch(epoch,lr):
    if epoch%8==0:
        return lr*0.96
    else:
        return lr
callback = tf.keras.callbacks.LearningRateScheduler(sch)

model.compile(optimizer=keras.optimizers.SGD(momentum=0.9,learning_rate=0.001),
             loss=[losses.sparse_categorical_crossentropy,
                   losses.sparse_categorical_crossentropy,
                   losses.sparse_categorical_crossentropy],
             loss_weights =[1,0.3,0.3],
             metrics=['accuracy'])
history = model.fit(train_generator,
                   batch_size=32,
                   epochs=40,
                   callbacks=[callback],
                   verbose=0)


Found 6000 images belonging to 2 classes.


2021-10-15 22:45:01.390698: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-10-15 22:45:01.980099: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-10-15 22:45:04.797183: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 777.72MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-10-15 22:45:04.916677: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 746.22MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-10-15 22:45:04.928736: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bf